In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [3]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [4]:
from HFCs_bmimPF6_PR import configuration

# Load data from csv
data = pd.read_csv('HFCSIL_g.csv')

In [5]:
def PR_model(data):
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(298.15)
    m.fs.state_block.pressure.fix(100000)
    m.fs.state_block.mole_frac_comp["R32"].fix(0.02)
    m.fs.state_block.mole_frac_comp["R125"].fix(0.02)
    m.fs.state_block.mole_frac_comp["bmimPF6"].fix(0.96)
    
    m.fs.properties.PR_kappa_A['R32', 'R125'].fix(-0.01)
    m.fs.properties.PR_kappa_A['R125', 'R32'].fix(-0.01)
    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.05450)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.10943)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(0.28636)
    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(-0.0035)
    
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix at actual temperature
    m.fs.state_block.temperature.fix(float(data["temperature"]))
#     m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp["R125"].unfix()
    m.fs.state_block.mole_frac_comp["R32"].unfix()
    m.fs.state_block.mole_frac_comp["bmimPF6"].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].fix(float(data["y_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].fix(float(data["y_R125"]))   
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data["x_R125"])) 
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimPF6'].fix(float(data["x_IL"])) 
    
    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['R32', 'R125'].setlb(-10)
    m.fs.properties.PR_kappa_A['R32', 'R125'].setub(10)

    m.fs.properties.PR_kappa_A['R125', 'R32'].setlb(-10)
    m.fs.properties.PR_kappa_A['R125', 'R32'].setub(10)

    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setlb(-10)
    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setub(10)

    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setlb(-10)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setub(10)
    
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setlb(-10)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setub(10)

    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setlb(-10)
    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setub(10)
    
    # Return initialized flash model
    return m

In [6]:
import pytest

test_data = {"temperature": 298.15, "pressure": 100000, "x_R32":0.7312, "x_IL":0.7312,  
             "x_R125":0.2688, "y_R125":0.2501, "y_R32":0.7499, "y_IL":0.7312}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

2021-09-02 13:44:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
The initial DOF is 1


In [7]:
variable_name = [
                "fs.properties.PR_kappa_A['R32', 'R125']",
                "fs.properties.PR_kappa_A['R125', 'R32']",
                "fs.properties.PR_kappa_A['R32', 'bmimPF6']",
                "fs.properties.PR_kappa_A['bmimPF6', 'R32']",
                "fs.properties.PR_kappa_A['bmimPF6', 'R125']",
                "fs.properties.PR_kappa_A['R125', 'bmimPF6']"]

In [8]:
def SSE(m, data):
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
    return expr*1e-9

# def SSE(m, data):
#     expr = ((float(data["y_R125"]) -
#              m.fs.state_block.mole_frac_phase_comp["Vap", "R125"])**2)


In [9]:
solver_options = {'tol': 1e-7}
pest = parmest.Estimator(PR_model, data, variable_name, SSE, solver_options)
# solver_options = {'tol': 1e-6, 'max_iter':100}

obj_value, parameters = pest.theta_est()

2021-09-02 13:44:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-02 13:44:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-02 13:44:43 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-02 13:44:45 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-02 13:44:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-02 13:44:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-02 13:44:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-02 13:44:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-02 13:44:56 [INFO] idaes

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  60  1.1223550e+01 1.93e-01 3.88e+06  -1.0 6.89e+00  -2.0 1.00e+00 8.32e-02h  4
  61  1.1223550e+01 1.90e-01 2.71e+06  -1.0 1.35e+01  -2.5 3.53e-01 1.05e-01h  4
  62  1.1223550e+01 1.84e-01 2.45e+06  -1.0 3.43e+01  -3.0 9.72e-01 5.31e-02h  5
  63  1.1223550e+01 1.83e-01 2.13e+06  -1.0 7.74e+01  -2.6 1.64e-01 9.63e-03f  4
  64  1.1223550e+01 1.88e-01 2.48e+06  -1.0 6.72e+01  -3.1 5.28e-01 2.44e-02h  6
  65  1.1223550e+01 2.01e-01 6.49e+06  -1.0 5.76e+00  -1.7 3.73e-01 6.72e-02h  3
  66  1.1223550e+01 2.15e-01 1.24e+07  -1.0 1.17e+01  -2.2 1.00e+00 5.22e-02h  4
  67  1.1223550e+01 2.29e-01 1.32e+07  -1.0 3.87e+01  -2.7 1.86e-01 3.92e-02h  5
  68  1.1223550e+01 2.38e-01 2.93e+07  -1.0 2.26e+01  -2.2 1.00e+00 3.46e-02h  5
  69  1.1223551e+01 2.47e-01 3.12e+07  -1.0 9.65e+01  -2.7 1.18e-01 1.72e-02h  6
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70  1.1223554e+01 3.01e+00

 153  1.1526017e+01 4.55e+00 3.83e+09  -1.0 4.86e-02   2.1 4.60e-01 3.71e-04h  1
 154  1.1526017e+01 4.55e+00 2.28e+11  -1.0 4.13e+02   1.6 3.14e-03 5.24e-05h  1
 155  1.1526017e+01 4.55e+00 4.10e+11  -1.0 4.46e+02   1.1 6.97e-03 2.92e-04f  5
 156  1.1526017e+01 4.56e+00 5.31e+11  -1.0 4.56e+02   0.6 1.38e-02 2.93e-04f  5
 157  1.1526017e+01 4.56e+00 5.45e+11  -1.0 4.81e+02   0.2 4.85e-03 1.51e-04f  6
 158  1.1526017e+01 4.56e+00 5.96e+11  -1.0 5.39e+02  -0.3 7.10e-02 3.27e-04f  5
 159  1.1526017e+01 4.56e+00 5.95e+11  -1.0 7.58e+02  -0.8 7.36e-03 4.97e-04f  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 160  1.1526019e+01 4.57e+00 4.29e+11  -1.0 1.01e+03  -1.3 5.01e-01 5.08e-03h  3
 161  1.1526031e+01 4.50e+00 4.20e+11  -1.0 1.14e+03  -1.7 6.34e-02 1.42e-02h  1
 162  1.1526069e+01 4.48e+00 4.69e+11  -1.0 3.81e+03  -2.2 1.76e-02 2.78e-03h  2
 163  1.1526925e+01 3.45e+00 2.56e+12  -1.0 1.98e+04  -2.7 2.44e-01 9.07e-03H  1
 164  1.1527257e+01 3.44e+00

 248  1.1570180e+01 2.18e+00 5.43e+09  -1.0 6.16e+02  -2.5 2.80e-02 3.59e-04h  6
 249  1.1570183e+01 2.18e+00 5.67e+10  -1.0 5.45e+02  -2.0 3.39e-01 9.13e-04h  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 250  1.1575950e+01 2.17e+00 5.24e+10  -1.0 1.06e+05    -  7.52e-02 1.13e-03h  5
 251  1.1668629e+01 2.14e+00 1.94e+10  -1.0 1.14e+05    -  6.20e-01 1.91e-02h  4
 252  1.1673023e+01 2.14e+00 1.88e+10  -1.0 3.48e+05    -  2.59e-02 3.02e-04h  7
 253  1.1676395e+01 2.13e+00 1.56e+10  -1.0 2.19e+05    -  1.61e-01 4.58e-04h  7
 254  1.2992668e+01 3.77e+00 7.16e+09  -1.0 1.13e+05    -  5.43e-01 3.66e-01w  1
 255  1.2993109e+01 3.74e+00 8.77e+09  -1.0 1.15e+03  -2.5 1.29e-02 9.74e-03w  1
 256  1.2996063e+01 3.71e+00 2.26e+10  -1.0 8.79e+03  -3.0 1.36e-02 6.88e-03w  1
 257  1.1788888e+01 2.04e+00 7.23e+09  -1.0 3.41e+04  -2.6 5.43e-01 4.57e-02h  3
 258  1.2112612e+01 1.81e+00 2.18e+09  -1.0 1.29e+05    -  7.52e-01 1.12e-01h  3
 259  1.1799462e+01 1.73e+00

 335  1.5989890e+00 4.16e-02 1.79e-01  -3.8 3.38e-01  -4.3 1.00e+00 1.00e+00h  1
 336  1.5989477e+00 1.50e-01 9.47e-01  -3.8 1.00e+00  -4.8 1.00e+00 1.00e+00h  1
 337  1.5989326e+00 2.59e-01 8.24e+01  -3.8 6.24e+00  -5.3 2.06e-01 1.22e-01h  3
 338  1.5989295e+00 2.81e-01 1.22e+03  -3.8 6.99e+00  -4.9 1.00e+00 6.75e-02h  4
 339  9.7936950e-01 5.12e+00 1.38e+03  -3.8 8.03e+04    -  3.00e-01 2.17e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 340  7.6794915e-01 9.49e+00 3.69e+04  -3.8 8.98e+04    -  2.18e-03 1.03e-01h  1
 341  7.6794821e-01 8.77e+00 3.49e+04  -3.8 2.51e+01  -4.4 5.46e-02 1.18e-01h  1
 342  7.6794657e-01 8.22e+00 1.66e+05  -3.8 8.21e+00  -4.9 5.54e-01 8.55e-02h  1
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
 343  7.6794175e-01 7.85e+00 8.17e+04  -3.8 2.75e+01  -5.4 3.01e-01 5.99e-02h  2
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
 344  7.6793481e-01 7.73e+00 2.8

 419  7.3546780e-05 1.90e+00 1.18e+01  -5.7 3.76e+02  -6.0 1.35e-01 1.97e-01h  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 420  7.3714505e-05 1.88e+00 1.14e+01  -5.7 3.48e+02  -6.5 9.15e-02 1.33e-02h  2
 421  1.2962654e-04 3.74e+00 1.73e+00  -5.7 3.52e+02  -7.0 4.89e-01 1.00e+00h  1
 422  1.2864136e-04 3.26e+00 6.89e+00  -5.7 2.28e+01  -6.5 1.00e+00 2.37e-01h  1
 423  1.2546036e-04 4.03e+00 2.34e+00  -5.7 7.89e+01  -7.0 8.13e-02 1.00e+00h  1
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
 424  1.2521530e-04 3.74e+00 6.22e-01  -5.7 7.70e+01  -7.5 4.61e-01 7.10e-02h  2
 425  1.2473805e-04 1.50e+00 1.68e+01  -5.7 3.89e+00  -6.2 1.64e-01 6.66e-01h  1
 426  1.2472121e-04 1.26e+00 1.57e+00  -5.7 2.24e+01  -5.7 4.98e-03 1.00e+00h  1
 427  1.2469931e-04 6.94e-01 3.99e-01  -5.7 8.06e+00  -5.3 6.14e-01 1.00e+00h  1
 428  1.2461793e-04 5.16e-01 6.72e-02  -5.7 6.13e+00  -5.8 1.00e+00 5.00e-01h  2
 429  1.2460840e-04 5.28e-01 6

 496  5.4417343e-06 5.44e-03 2.85e-06  -8.6 9.06e-01  -8.6 1.00e+00 5.00e-01h  2
 497  5.4421438e-06 5.67e-03 2.44e-06  -8.6 9.00e-01  -8.2 1.00e+00 1.25e-01h  4
 498  5.4425032e-06 6.06e-03 2.05e-06  -8.6 6.42e-01  -7.8 1.00e+00 1.25e-01h  4
 499  5.4437626e-06 5.79e-03 1.01e-06  -8.6 4.21e-02  -6.4 1.00e+00 5.00e-01h  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 500  5.4450218e-06 6.12e-03 1.03e-07  -8.6 5.63e-02  -6.9 1.00e+00 1.00e+00h  1
 501  5.4450240e-06 1.52e-03 3.26e-08  -8.6 4.18e-02  -7.4 1.00e+00 1.00e+00h  1
 502  5.4450188e-06 7.53e-03 2.89e-08  -8.6 5.97e-02  -7.9 1.00e+00 1.00e+00h  1
 503  5.4449998e-06 9.73e-03 5.11e-08  -8.6 7.37e-01  -8.3 1.00e+00 1.00e+00H  1
 504  5.4448819e-06 9.68e-04 2.44e-08  -8.6 4.34e-01  -8.8 1.00e+00 1.00e+00h  1
 505  5.4446867e-06 2.72e-06 1.73e-09  -8.6 5.08e-02  -7.5 1.00e+00 1.00e+00h  1
 506  1.7593298e-06 1.00e+00 1.61e-03  -8.6 1.55e+00  -8.0 6.65e-01 5.43e-01H  1
 507  1.8068392e-06 9.77e-01

In [10]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 0.000005

The values for the parameters are as follows:
fs.properties.PR_kappa_A[R32,R125] = -0.09826271426131736
fs.properties.PR_kappa_A[R125,R32] = -0.057981396806376394
fs.properties.PR_kappa_A[R32,bmimPF6] = -0.06350062845688859
fs.properties.PR_kappa_A[bmimPF6,R32] = -0.4022025106760566
fs.properties.PR_kappa_A[bmimPF6,R125] = 1.5882264494871066
fs.properties.PR_kappa_A[R125,bmimPF6] = -0.0010739298820285946


In [11]:
print(large_residuals_set(pest.ef_instance))

ComponentSet([])
